Import Libs

In [21]:
%matplotlib inline

import torch
import matplotlib.pyplot as plt
import requests
import os
import zipfile
import pandas as pd
from tqdm.auto import tqdm
from numpy import isnan
from datetime import timedelta

Download Data

In [4]:
if not os.path.exists('data/electricity.zip'):
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip'
    response = requests.get(url, allow_redirects=True)

    with open("data/electricity.zip","wb") as handle:
        for data in tqdm(response.iter_content()):
            handle.write(data)

    with zipfile.ZipFile("data/electricity.zip",'r') as zip_ref:
        zip_ref.extractall("data")

    os.rename('data/household_power_consumption.txt','data/household_power_consumption.csv')


Preprocess Data

In [80]:
resolution = 60
feature_dim = 24*60//resolution
df_raw = pd.read_csv('data/household_power_consumption.csv', sep=';', index_col=False, low_memory=False, na_values='?')
df_raw = df_raw.iloc[21996::resolution,:3]
df_raw['datetime'] = pd.to_datetime(df_raw['Date']+' '+df_raw['Time'], infer_datetime_format=True, dayfirst=True)
df_raw = df_raw.drop(['Date','Time'],axis=1)
df_raw = df_raw.set_index(['datetime'])

In [81]:
g = df_raw.groupby(df_raw.index.floor('d'))

daily_data = []
my_day, final_day = df_raw.index[0].date(), df_raw.index[-1].date()

while my_day < final_day:
    try:
        day_data = g.get_group(my_day).T.values.tolist()[0]
    except:
        print("Date "+str(my_day)+" is missing.")
        my_day += timedelta(days=1)
    else:
        if (not isnan(day_data).any()) and day_data.__len__() == feature_dim:
            row_data = [my_day.month, my_day.weekday()] + day_data
            daily_data.append(row_data)
        my_day += timedelta(days=1)

print('Dataset Size:'+str(daily_data.__len__()))



Dataset Size:1396


In [83]:
column_names = ['month', 'weekday']
my_day = pd.Timestamp('2012-01-01')
for _ in range(feature_dim):
    column_names.append(my_day.strftime('%H:%M'))
    my_day+=timedelta(minutes=resolution)

# %%
df_daily = pd.DataFrame(daily_data,columns=column_names)
df_daily.to_csv('data/daily_data.csv',index=False)

Split Dataset

In [84]:
from sklearn.model_selection import train_test_split
train_ratio = 0.8
df_train, df_test = train_test_split(df_daily, train_size=train_ratio)

DataLoader

In [85]:
class ElectricityData(torch.utils.data.Dataset):
    def __init__(self, df):
        data = torch.tensor(df.values).float()
        self.inputs = data[:,2:]
        self.mean = self.inputs.mean(dim=0)
        self.std = self.inputs.std(dim=0)
        self.inputs = (self.inputs-self.mean)/self.std

        self.conditions = self.circlize(data[:,[[0],[1]]])

    def circlize(self,condition):
        max_conds = torch.max(condition,dim=0,keepdim=False)[0]
        min_conds = torch.min(condition,dim=0,keepdim=False)[0]
        return torch.cat((torch.cos((condition-min_conds)/(max_conds-min_conds+1)*2*torch.pi),torch.sin((condition-min_conds)/(max_conds-min_conds+1)*2*torch.pi)),dim=2).reshape(condition.shape[0],-1)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        return self.inputs[idx], self.conditions[idx]


Neural Networks

In [88]:
NUM_NEURONS = 50
ACTIVATION = torch.nn.CELU(inplace=True)

class Encoder(torch.nn.Module):
    def __init__(self,input_dim,latent_dim):
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim

        self.fc1 = torch.nn.Linear(input_dim, NUM_NEURONS)
        self.fc2 = torch.nn.Linear(NUM_NEURONS, NUM_NEURONS)
        self.fc_mean = torch.nn.Linear(NUM_NEURONS, latent_dim)
        self.fc_sigmatilde = torch.nn.Linear(NUM_NEURONS, latent_dim)

        # setup the non-linearity
        self.act = ACTIVATION

    def forward(self, x):
        h = x.view(-1, self.input_dim)
        h = self.act(self.fc1(h))
        h = self.act(self.fc2(h))
        z_mean = self.fc_mean(h)
        z_sigmatilde = self.fc_sigmatilde(h)
        return z_mean, z_sigmatilde

    def _num_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

class Decoder(torch.nn.Module):
    def __init__(self,output_dim,latent_dim,learn_sigma=False):
        super(Decoder, self).__init__()
        self.input_dim = output_dim
        self.latent_dim = latent_dim
        self.learn_sigma = learn_sigma

        self.fc1 = torch.nn.Linear(latent_dim, NUM_NEURONS)
        self.fc2 = torch.nn.Linear(NUM_NEURONS, NUM_NEURONS)
        self.fc_mean = torch.nn.Linear(NUM_NEURONS, output_dim)
        if learn_sigma: self.fc_sigmatilde = torch.nn.Linear(NUM_NEURONS, output_dim)

        # setup the non-linearity
        self.act = ACTIVATION

    def forward(self, z):
        h = z.view(-1, self.latent_dim)
        h = self.act(self.fc1(h))
        h = self.act(self.fc2(h))
        x_mean = self.fc_mean(h)
        if self.learn_sigma:
            x_sigmatilde = self.fc_sigmatilde(h)
        else:
            x_sigmatilde = 0.0*x_mean
        return x_mean, x_sigmatilde

    def _num_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

VAE Model

In [87]:
class VAE(torch.nn.Module):
    def __init__(self,input_dim=24, cond_dim=4, latent_dim=10, learn_sigma=True):
        super(VAE, self).__init__()
        
        self.input_dim, self.cond_dim, self.latent_dim = input_dim, cond_dim, latent_dim

        self.encoder = Encoder(self.input_dim+self.cond_dim, self.latent_dim)
        self.decoder = Decoder(self.input_dim, self.latent_dim+self.cond_dim, learn_sigma)
        
        self.num_parameters = self.encoder._num_parameters()+self.decoder._num_parameters()
    
    def forward(self,inputs,conditions):
        z_mean, z_sigmatilde = self.encoder(torch.cat((inputs,conditions),dim=1))
        z = self.sample(z_mean, z_sigmatilde)
        x_mean, x_sigmatilde = self.decoder(torch.cat((z,conditions),dim=1))
        return {"mean": x_mean, "sigmatilde": x_sigmatilde}, {"mean": z_mean, "sigmatilde": z_sigmatilde, "sample": z} 

    def sample(self,mean,sigmatilde):
        eps = torch.randn(mean.shape)
        return mean + eps*self.to_sigma(sigmatilde)

    def reconstruction_loglikelihood(self, input, mean, sigmatilde):
        sigma = self.to_sigma(sigmatilde)
        return (-.5*(torch.tensor(2)*torch.pi).log()-sigma.log()-.5*((input-mean)/sigma).pow(2)).sum(dim=1)

    def kl_divergence(self, mean_posterior, sigmatilde_posterior, mean_prior=None, sigmatilde_prior=None):
        if mean_prior is None: mean_prior = mean_posterior*0.0
        if sigmatilde_prior is None: sigmatilde_prior =  self.from_sigma(sigmatilde_posterior*0.0 + 1.0)

        sigma_posterior = self.to_sigma(sigmatilde_posterior)
        sigma_prior = self.to_sigma(sigmatilde_prior)

        return .5*((sigma_posterior/sigma_prior).pow(2) + ((mean_posterior-mean_prior)/sigma_prior).pow(2) -1 + 2*(sigma_prior.log()-sigma_posterior.log())).sum(dim=1)
    
    def loss(self, input, x_mean, x_sigmatilde, z_mean, z_sigmatilde, beta=1):
        rll = self.reconstruction_loglikelihood(input,x_mean,x_sigmatilde).mean(dim=0)
        kl = self.kl_divergence(z_mean,z_sigmatilde).mean(dim=0)
        return {"loss":-(rll-beta*kl), "elbo": rll-kl, "rll": rll, "kl": kl}

    def to_sigma(self,sigmatilde):
        return torch.log2(1+torch.pow(torch.tensor(2),sigmatilde))

    def from_sigma(self,sigma):
        return torch.log2(torch.pow(torch.tensor(2),sigma)-1)

Training Settings

In [86]:
batch_size = 32
epochs = 1000
beta = 1
learning_rate = 1e-3
latent_dim = 2
verbose_freq = 50

Training Loop

In [ ]:
dset = ElectricityData(df_train)
train_loader = torch.utils.data.DataLoader(dset,batch_size=32,shuffle=True,drop_last=True)
model = VAE(input_dim=feature_dim, cond_dim=4, latent_dim=latent_dim)

pbar = tqdm(total=(dset.__len__()//batch_size+1) * epochs)

optim = torch.optim.Adam([{'params':model.encoder.parameters()},
                        {'params':model.decoder.parameters()}],lr=learning_rate)

epx = 0
itx = 0
for _ in range(epochs):
    epx += 1
    for inputs, conditions in train_loader:
        itx += 1
        pbar.update(1)
        x_dict, z_dict = model.forward(inputs, conditions)

        optim.zero_grad()
        loss = model.loss(inputs,x_dict["mean"],x_dict["sigmatilde"],z_dict["mean"],z_dict["sigmatilde"])
        loss["loss"].backward()
        optim.step()

        if itx%verbose_freq==0: 
            pbar.write(f"Iteration: {itx} -- RLL={loss['rll'].item():.4f} / KL={loss['kl'].item():.4f}")
            pbar.write(f"Min_sigmatilde: {x_dict['sigmatilde'].min().item()}")


In [ ]:

    #End of Epoch
    with torch.no_grad():
        
        #region Train Viz
        #TODO: Shuffle=False is needed rn. Find a way to keep same training sample
        denorm_inputs = denormalize(inputs[0],stats['mean'],stats['vh'],stats['std'])
        denorm_x = denormalize(x_dict['mean'][0],stats['mean'],stats['vh'],stats['std'])
        denorm_sigma = denormalize_sigma(x_dict['sigmatilde'][0],stats['vh'],stats['std'])

        #region Viz block
        fig, ax = viz.compare_with_bounds(denorm_inputs,denorm_x,denorm_sigma,fig_no=1)
        ax.set_title(f"Training Sample (Epoch: {epx:03d})")
        ax.set_ylabel("Consumption [kWh]")
        ax.set_xticks(list(range(0,48,6)),get_timeslots(30*6))
        if self.savefig: 
            savedir = self.log_dir+"/train_imgs"
            if not os.path.isdir(savedir):
                os.makedirs(savedir)
            with open(savedir+"/makegif.sh",'w') as f:
                f.write("convert -delay 10 -loop 0 *.png myimage.gif")
            plt.savefig(savedir+f"/{epx:03d}")
        #endregion
        #endregion

        #region Validation
        x_dict, z_dict = self.forward(**valset)
        inputs = valset["inputs"]

        denorm_inputs = denormalize(inputs,stats['mean'],stats['vh'],stats['std'])
        denorm_x = denormalize(x_dict['mean'],stats['mean'],stats['vh'],stats['std'])
        denorm_sigma = denormalize_sigma(x_dict['sigmatilde'],stats['vh'],stats['std'])

        rmse = (denorm_inputs-denorm_x).pow(2).mean()
        loss = self.loss(inputs,x_dict["mean"],x_dict["sigmatilde"],z_dict["mean"],z_dict["sigmatilde"]) 
        self.pbar.write(f"Validation -- RLL={loss['rll'].item():.4f} / KL={loss['kl'].item():.4f}")
        self.pbar.write(f"Validation -- RMSE={rmse.item():.4f}")

        #region Viz block
        idx = 0
        fig, ax = viz.compare_with_bounds(denorm_inputs[idx],denorm_x[idx],denorm_sigma[idx],fig_no=2)
        ax.set_title(f"Validation Sample (Epoch: {epx:03d})")
        ax.set_ylabel("Consumption [kWh]")
        ax.set_xticks(list(range(0,48,6)),get_timeslots(30*6))
        if self.savefig: 
            savedir = self.log_dir+"/val_imgs"
            if not os.path.isdir(savedir):
                os.makedirs(savedir)
            with open(savedir+"/makegif.sh",'w') as f:
                f.write("convert -delay 10 -loop 0 *.png myimage.gif")
            plt.savefig(savedir+f"/{epx:03d}")
        #endregion

        plt.show(block=False)
        plt.pause(0.1)
        #endregion

        #region Tensorboard Visualization
        if self.viz:
            if self.viz: 
                self.writer.add_scalars('Loss/RLL', {'val':loss['rll']}, itx)
                self.writer.add_scalars('Loss/KL', {'val':loss['kl']}, itx)
        #endregion